[Reference](https://ridgerunai.medium.com/machine-learning-mean-average-precision-map-and-other-object-detection-metrics-45267507a904)

In [ ]:
def iou(bbox_a, bbox_b):
    ax1, ay1, ax2, ay2 = bbox_a
    bx1, by1, bx2, by2 = bbox_b

    # Compute the coordinates of the intersection
    ix1 = max(ax1, bx1)
    iy1 = max(ay1, by1)
    ix2 = min(ax2, bx2)
    iy2 = min(ay2, by2)

    # Compute the area of intersection rectangle
    intersection = max(0, ix2 - ix1) * max(0, iy2 - iy1)

    # Compute the area of both bounding boxes
    box1_area = (ax2 - ax1) * (ay2 - ay1)
    box2_area = (bx2 - bx1) * (by2 - by1)

    # Finally compute the union of the areas
    union = box1_area + box2_area - intersection


    return intersection / union

In [ ]:
!pip3 install ultralytics
!pip3 install requests
!pip3 install pycocotools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 757.0/757.0 kB 7.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [ ]:
# To download a sample dataset
import requests
import zipfile
from tqdm import tqdm

# An example model
from ultralytics import YOLO

# COCO tools
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval

# To check if we need to re-download
import os

# Helper to download files
def download_file(url, file_path):
    response = requests.get(url, stream=True)
    total_size_in_bytes = int(response.headers.get('content-length', 0))
    block_size = 1024  # 1 Kibibyte

    progress_bar = tqdm(total=total_size_in_bytes, unit='iB', unit_scale=True)

    with open(file_path, 'wb') as file:
        for data in response.iter_content(block_size):
            progress_bar.update(len(data))
            file.write(data)
    progress_bar.close()

coco_url = "https://github.com/ultralytics/yolov5/releases/download/v1.0/coco2017val.zip"
coco_zip_path = 'coco2017val.zip'

# Only download and unzip if the coco directory is not there
if not os.path.exists('coco'):
    download_file(coco_url, coco_zip_path)

    # Unzip the file
    with zipfile.ZipFile(coco_zip_path, 'r') as zip_ref:
        zip_ref.extractall('.')

# Load pre-trained YOLOv8 model
model = YOLO('./yolov8n.pt')

# Load COCO validation images annotations
coco_annotations_path = 'coco/annotations/instances_val2017.json'
coco = COCO(coco_annotations_path)

# Get image IDs
image_ids = coco.getImgIds()
images = coco.loadImgs(image_ids)

# Process images and collect detections
results = []
for img in images:
    image_path = f"coco/images/val2017/{img['file_name']}"
    preds = model(image_path)[0].numpy().boxes

    # Convert results to a COCO compatible format
    for xyxy, conf, cls in zip(preds.xyxy, preds.conf, preds.cls):
        result = {
            'image_id': img['id'],
            'category_id': int(cls.item()+1),
            'bbox': [xyxy[0].item(), xyxy[1].item(), xyxy[2].item() - xyxy[0].item(), xyxy[3].item() - xyxy[1].item()],
            'score': conf.item()
        }

        results.append(result)

# Convert results to COCO object
coco_dt = coco.loadRes(results)

# Run COCO evaluation
coco_eval = COCOeval(coco, coco_dt, 'bbox')
coco_eval.evaluate()
coco_eval.accumulate()
coco_eval.summarize()

100%|██████████| 818M/818M [00:18<00:00, 45.3MiB/s]


100%|██████████| 6.23M/6.23M [00:00<00:00, 98.3MB/s]


Streaming output truncated to the last 5000 lines.
image 1/1 /content/coco/images/val2017/000000433204.jpg: 480x640 5 persons, 2 cars, 1 motorcycle, 174.8ms
Speed: 2.6ms preprocess, 174.8ms inference, 1.4ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 /content/coco/images/val2017/000000245448.jpg: 480x640 5 persons, 1 car, 4 motorcycles, 1 truck, 185.5ms
Speed: 2.2ms preprocess, 185.5ms inference, 1.3ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 /content/coco/images/val2017/000000213445.jpg: 640x544 1 cat, 1 bowl, 1 chair, 2 couchs, 1 dining table, 2 books, 194.4ms
Speed: 3.3ms preprocess, 194.4ms inference, 1.6ms postprocess per image at shape (1, 3, 640, 544)

image 1/1 /content/coco/images/val2017/000000224724.jpg: 640x480 2 persons, 2 cars, 2 traffic lights, 1 handbag, 200.5ms
Speed: 9.8ms preprocess, 200.5ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 480)

image 1/1 /content/coco/images/val2017/000000163155.jpg: 448x640 1 cat, 1 suitc